In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import re
import nltk
from nltk.corpus import stopwords

from nltk.stem import WordNetLemmatizer

from nltk.tokenize import sent_tokenize,word_tokenize


from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB,MultinomialNB,BernoulliNB

In [68]:
df=pd.read_csv('D:\Brand Sentiment Analysis\Dataset\Dataset - Train.csv')
df.head()

,tweet_text,emotion_in_tweet_is_directed_at,is_there_an_emotion_directed_at_a_brand_or_product
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,iPhone,Negative emotion
1,@jessedee Know about @fludapp ? Awesome iPad/i...,iPad or iPhone App,Positive emotion
2,@swonderlin Can not wait for #iPad 2 also. The...,iPad,Positive emotion
3,@sxsw I hope this year's festival isn't as cra...,iPad or iPhone App,Negative emotion
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,Google,Positive emotion


In [69]:
df=df.dropna()

In [70]:
df['is_there_an_emotion_directed_at_a_brand_or_product'].value_counts()

is_there_an_emotion_directed_at_a_brand_or_product
Positive emotion                      2672
Negative emotion                       519
No emotion toward brand or product      91
I can't tell                             9
Name: count, dtype: int64

In [71]:
df.rename(columns={'is_there_an_emotion_directed_at_a_brand_or_product':'emotion',
                   'tweet_text':'text'},inplace=True)

In [72]:
df.head()

,text,emotion_in_tweet_is_directed_at,emotion
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,iPhone,Negative emotion
1,@jessedee Know about @fludapp ? Awesome iPad/i...,iPad or iPhone App,Positive emotion
2,@swonderlin Can not wait for #iPad 2 also. The...,iPad,Positive emotion
3,@sxsw I hope this year's festival isn't as cra...,iPad or iPhone App,Negative emotion
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,Google,Positive emotion


In [73]:
def remove_tags(text):
    remove=re.compile(r'')
    return re.sub(remove, '', text)

df['text'] = df['text'].apply(remove_tags)

In [74]:
df.head()

,text,emotion_in_tweet_is_directed_at,emotion
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,iPhone,Negative emotion
1,@jessedee Know about @fludapp ? Awesome iPad/i...,iPad or iPhone App,Positive emotion
2,@swonderlin Can not wait for #iPad 2 also. The...,iPad,Positive emotion
3,@sxsw I hope this year's festival isn't as cra...,iPad or iPhone App,Negative emotion
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,Google,Positive emotion


In [75]:
def remove_splchr(text):
    cp=[]
    text=text.split()
    
    for i in text:
        if i.isalnum():
            cp.append(i)
        
    return " ".join(cp)

df['text']=df['text'].apply(remove_splchr)
    

In [76]:
df['text']=df['text'].str.lower()

In [77]:
def remove_stopwords(text):
    stopword=set(stopwords.words('english'))
    text=word_tokenize(text)
    
    return " ".join([x for x in text if x not in stopword])

# df['text']=df['text'].apply(remove_stopwords)

In [78]:
def lemmatize_word(text):
  wordnet = WordNetLemmatizer()
  return " ".join([wordnet.lemmatize(word) for word in text.split()])

df['text']=df['text'].apply(lemmatize_word)

In [79]:
df.head()

,text,emotion_in_tweet_is_directed_at,emotion
0,i have a 3g after 3 hr tweeting at it wa i nee...,iPhone,Negative emotion
1,know about awesome app that likely appreciate ...,iPad or iPhone App,Positive emotion
2,can not wait for 2 they should sale them down at,iPad,Positive emotion
3,i hope this festival a crashy a this iphone,iPad or iPhone App,Negative emotion
4,great stuff on fri marissa mayer tim matt mull...,Google,Positive emotion


In [80]:
df['text']=df['emotion_in_tweet_is_directed_at']+ " " + df['text']

In [81]:
df.head()

,text,emotion_in_tweet_is_directed_at,emotion
0,iPhone i have a 3g after 3 hr tweeting at it w...,iPhone,Negative emotion
1,iPad or iPhone App know about awesome app that...,iPad or iPhone App,Positive emotion
2,iPad can not wait for 2 they should sale them ...,iPad,Positive emotion
3,iPad or iPhone App i hope this festival a cras...,iPad or iPhone App,Negative emotion
4,Google great stuff on fri marissa mayer tim ma...,Google,Positive emotion


In [82]:
df['emotion'].unique()

array(['Negative emotion', 'Positive emotion',
       'No emotion toward brand or product', "I can't tell"], dtype=object)

In [83]:
df.rename(index={'Negative emotion':0,'Positive emotion':1,'No emotion toward brand or product':2,"I can't tell":3},inplace=True)

In [84]:
df.head()

,text,emotion_in_tweet_is_directed_at,emotion
0,iPhone i have a 3g after 3 hr tweeting at it w...,iPhone,Negative emotion
1,iPad or iPhone App know about awesome app that...,iPad or iPhone App,Positive emotion
2,iPad can not wait for 2 they should sale them ...,iPad,Positive emotion
3,iPad or iPhone App i hope this festival a cras...,iPad or iPhone App,Negative emotion
4,Google great stuff on fri marissa mayer tim ma...,Google,Positive emotion


In [85]:
X=df['text']
y=df['emotion']

In [86]:
X=np.array(X.values)

In [87]:
y=np.array(y.values)

In [89]:
cv=CountVectorizer(max_features=5000)
X=cv.fit_transform(df['text']).toarray()

In [91]:
print("X.shape : ",X.shape)
print("y.value : ",len(y))

X.shape :  (3291, 3905)
y.value :  3291


In [92]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0, shuffle = True)
print(len(X_train))
print(len(X_test))

2303
988


In [94]:
from sklearn.metrics import precision_recall_fscore_support as score

In [95]:
LR=LogisticRegression()
LR.fit(X_train,y_train)
y_pred=LR.predict(X_test)
precision, recall, f1score, support = score(y_test, y_pred, average='micro')
print(accuracy_score(y_test,y_pred))
print(precision)
print(recall)
print(f1score)

0.8502024291497976
0.8502024291497976
0.8502024291497976
0.8502024291497976


d:\Brand Sentiment Analysis\venv\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [98]:
RNN=RandomForestClassifier(n_estimators=100,criterion='entropy',random_state=2)
RNN.fit(X_train,y_train)
y_pred=RNN.predict(X_test)
precision, recall, f1score, support = score(y_test, y_pred, average='micro')
print(accuracy_score(y_test,y_pred))
print(precision)
print(recall)
print(f1score)

0.847165991902834
0.847165991902834
0.847165991902834
0.8471659919028341


In [99]:
MNB = MultinomialNB(alpha=1.0,fit_prior=True)

MNB.fit(X_train,y_train)
y_pred=MNB.predict(X_test)
precision, recall, f1score, support = score(y_test, y_pred, average='micro')
print(accuracy_score(y_test,y_pred))
print(precision)
print(recall)
print(f1score)

0.840080971659919
0.840080971659919
0.840080971659919
0.840080971659919


In [116]:
test2="#SXSW is just starting, #CTIA is around the corner and #googleio is only a hop skip and a jump from there, good time to be an #android fan"

test2=remove_tags(test2)
test2=remove_splchr(test2)
test2=test2.lower()
test2=lemmatize_word(test2)




y_pred1 = cv.transform([test2])

yy =MNB.predict(y_pred1)

print(yy)

['Positive emotion']


In [115]:
test2=".@wesley83 I have a 3G iPhone. After 3 hrs tweeting at #RISE_Austin, it was dead!  I need to upgrade. Plugin stations at #SXSW."

test2=remove_tags(test2)
test2=remove_splchr(test2)
test2=test2.lower()
test2=lemmatize_word(test2)




y_pred1 = cv.transform([test2])

yy =LR.predict(y_pred1)

print(yy)

['Negative emotion']
